In [1]:
import numpy as np
import pandas as pd
import random
import pickle
from nltk.tokenize import RegexpTokenizer
from tensorflow.keras.models import Sequential,load_model
from tensorflow.keras.layers import LSTM ,Dense ,Activation
from tensorflow.keras.optimizers import RMSprop



In [3]:
!unzip '/content/FAKE AND REAL NEWS DATASET.zip' -d '/content'

Archive:  /content/FAKE AND REAL NEWS DATASET.zip
  inflating: /content/Fake.csv       
  inflating: /content/True.csv       


In [2]:
!pip install pandas


In [4]:
text_df = pd.read_csv("/content/Fake.csv")
print(text_df.head())


                                               title  \
0   Donald Trump Sends Out Embarrassing New Year’...   
1   Drunk Bragging Trump Staffer Started Russian ...   
2   Sheriff David Clarke Becomes An Internet Joke...   
3   Trump Is So Obsessed He Even Has Obama’s Name...   
4   Pope Francis Just Called Out Donald Trump Dur...   

                                                text subject  \
0  Donald Trump just couldn t wish all Americans ...    News   
1  House Intelligence Committee Chairman Devin Nu...    News   
2  On Friday, it was revealed that former Milwauk...    News   
3  On Christmas day, Donald Trump announced that ...    News   
4  Pope Francis used his annual Christmas Day mes...    News   

                date  
0  December 31, 2017  
1  December 31, 2017  
2  December 30, 2017  
3  December 29, 2017  
4  December 25, 2017  


In [5]:
text_df

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"
...,...,...,...,...
23476,McPain: John McCain Furious That Iran Treated ...,21st Century Wire says As 21WIRE reported earl...,Middle-east,"January 16, 2016"
23477,JUSTICE? Yahoo Settles E-mail Privacy Class-ac...,21st Century Wire says It s a familiar theme. ...,Middle-east,"January 16, 2016"
23478,Sunnistan: US and Allied ‘Safe Zone’ Plan to T...,Patrick Henningsen 21st Century WireRemember ...,Middle-east,"January 15, 2016"
23479,How to Blow $700 Million: Al Jazeera America F...,21st Century Wire says Al Jazeera America will...,Middle-east,"January 14, 2016"


In [6]:
text_df = pd.read_csv("/content/True.csv")
print(text_df.head())


                                               title  \
0  As U.S. budget fight looms, Republicans flip t...   
1  U.S. military to accept transgender recruits o...   
2  Senior U.S. Republican senator: 'Let Mr. Muell...   
3  FBI Russia probe helped by Australian diplomat...   
4  Trump wants Postal Service to charge 'much mor...   

                                                text       subject  \
0  WASHINGTON (Reuters) - The head of a conservat...  politicsNews   
1  WASHINGTON (Reuters) - Transgender people will...  politicsNews   
2  WASHINGTON (Reuters) - The special counsel inv...  politicsNews   
3  WASHINGTON (Reuters) - Trump campaign adviser ...  politicsNews   
4  SEATTLE/WASHINGTON (Reuters) - President Donal...  politicsNews   

                 date  
0  December 31, 2017   
1  December 29, 2017   
2  December 31, 2017   
3  December 30, 2017   
4  December 29, 2017   


In [7]:
text_df

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"
...,...,...,...,...
21412,'Fully committed' NATO backs new U.S. approach...,BRUSSELS (Reuters) - NATO allies on Tuesday we...,worldnews,"August 22, 2017"
21413,LexisNexis withdrew two products from Chinese ...,"LONDON (Reuters) - LexisNexis, a provider of l...",worldnews,"August 22, 2017"
21414,Minsk cultural hub becomes haven from authorities,MINSK (Reuters) - In the shadow of disused Sov...,worldnews,"August 22, 2017"
21415,Vatican upbeat on possibility of Pope Francis ...,MOSCOW (Reuters) - Vatican Secretary of State ...,worldnews,"August 22, 2017"


In [8]:
text = list(text_df.text.values)
joined_text = "".join(text)


In [9]:
partial_text = joined_text[:1000]

In [45]:
partial_text = "He will have to look into this thing and he will be a man."


In [10]:

tokenizer = RegexpTokenizer(r'\w+')
tokens = tokenizer.tokenize(partial_text.lower())
print(tokens)

['washington', 'reuters', 'the', 'head', 'of', 'a', 'conservative', 'republican', 'faction', 'in', 'the', 'u', 's', 'congress', 'who', 'voted', 'this', 'month', 'for', 'a', 'huge', 'expansion', 'of', 'the', 'national', 'debt', 'to', 'pay', 'for', 'tax', 'cuts', 'called', 'himself', 'a', 'fiscal', 'conservative', 'on', 'sunday', 'and', 'urged', 'budget', 'restraint', 'in', '2018', 'in', 'keeping', 'with', 'a', 'sharp', 'pivot', 'under', 'way', 'among', 'republicans', 'u', 's', 'representative', 'mark', 'meadows', 'speaking', 'on', 'cbs', 'face', 'the', 'nation', 'drew', 'a', 'hard', 'line', 'on', 'federal', 'spending', 'which', 'lawmakers', 'are', 'bracing', 'to', 'do', 'battle', 'over', 'in', 'january', 'when', 'they', 'return', 'from', 'the', 'holidays', 'on', 'wednesday', 'lawmakers', 'will', 'begin', 'trying', 'to', 'pass', 'a', 'federal', 'budget', 'in', 'a', 'fight', 'likely', 'to', 'be', 'linked', 'to', 'other', 'issues', 'such', 'as', 'immigration', 'policy', 'even', 'as', 'the'

In [11]:
unique_tokens = np.unique(tokens)
unique_tokens_index = {token: idx for idx, token in enumerate(unique_tokens )}

In [12]:
unique_tokens_index

{'2018': 0,
 'a': 1,
 'also': 2,
 'among': 3,
 'and': 4,
 'approach': 5,
 'are': 6,
 'as': 7,
 'battle': 8,
 'be': 9,
 'begin': 10,
 'big': 11,
 'bracing': 12,
 'budget': 13,
 'called': 14,
 'campaigns': 15,
 'cbs': 16,
 'congress': 17,
 'congressional': 18,
 'conservative': 19,
 'control': 20,
 'cuts': 21,
 'debt': 22,
 'defense': 23,
 'democrats': 24,
 'discretionary': 25,
 'do': 26,
 'donald': 27,
 'drew': 28,
 'educati': 29,
 'election': 30,
 'even': 31,
 'expansion': 32,
 'face': 33,
 'faction': 34,
 'federal': 35,
 'fight': 36,
 'fiscal': 37,
 'for': 38,
 'from': 39,
 'hard': 40,
 'head': 41,
 'himself': 42,
 'his': 43,
 'holidays': 44,
 'huge': 45,
 'immigration': 46,
 'in': 47,
 'increase': 48,
 'increases': 49,
 'issues': 50,
 'january': 51,
 'keep': 52,
 'keeping': 53,
 'lawmakers': 54,
 'likely': 55,
 'line': 56,
 'linked': 57,
 'mark': 58,
 'meadows': 59,
 'military': 60,
 'month': 61,
 'nation': 62,
 'national': 63,
 'non': 64,
 'november': 65,
 'of': 66,
 'on': 67,
 'othe

In [13]:
n_words = 15
input_words = []
next_words = []
for i in range(len(tokens) - n_words):
    input_seq = tokens[i:i + n_words]
    next_word = tokens[i + n_words]
    input_words.append(input_seq)
    next_words.append(next_word)

In [14]:
next_words

['voted',
 'this',
 'month',
 'for',
 'a',
 'huge',
 'expansion',
 'of',
 'the',
 'national',
 'debt',
 'to',
 'pay',
 'for',
 'tax',
 'cuts',
 'called',
 'himself',
 'a',
 'fiscal',
 'conservative',
 'on',
 'sunday',
 'and',
 'urged',
 'budget',
 'restraint',
 'in',
 '2018',
 'in',
 'keeping',
 'with',
 'a',
 'sharp',
 'pivot',
 'under',
 'way',
 'among',
 'republicans',
 'u',
 's',
 'representative',
 'mark',
 'meadows',
 'speaking',
 'on',
 'cbs',
 'face',
 'the',
 'nation',
 'drew',
 'a',
 'hard',
 'line',
 'on',
 'federal',
 'spending',
 'which',
 'lawmakers',
 'are',
 'bracing',
 'to',
 'do',
 'battle',
 'over',
 'in',
 'january',
 'when',
 'they',
 'return',
 'from',
 'the',
 'holidays',
 'on',
 'wednesday',
 'lawmakers',
 'will',
 'begin',
 'trying',
 'to',
 'pass',
 'a',
 'federal',
 'budget',
 'in',
 'a',
 'fight',
 'likely',
 'to',
 'be',
 'linked',
 'to',
 'other',
 'issues',
 'such',
 'as',
 'immigration',
 'policy',
 'even',
 'as',
 'the',
 'november',
 'congressional',
 

In [15]:
input_words

[['washington',
  'reuters',
  'the',
  'head',
  'of',
  'a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who'],
 ['reuters',
  'the',
  'head',
  'of',
  'a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who',
  'voted'],
 ['the',
  'head',
  'of',
  'a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who',
  'voted',
  'this'],
 ['head',
  'of',
  'a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who',
  'voted',
  'this',
  'month'],
 ['of',
  'a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who',
  'voted',
  'this',
  'month',
  'for'],
 ['a',
  'conservative',
  'republican',
  'faction',
  'in',
  'the',
  'u',
  's',
  'congress',
  'who',
  'voted',
  'this',
  'month',
  'for',
  'a'],
 ['conservative',
  'republican',
  'faction

In [16]:
X = np.zeros((len(input_words), n_words, len(unique_tokens)), dtype=bool)
Y = np.zeros((len(next_words), len(unique_tokens)), dtype=bool)




In [17]:
X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, Fal

In [18]:
Y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [19]:
for i , words  in enumerate(input_words):
  for j , word in enumerate(words):
    X[i,j, unique_tokens_index[word]] = 2
    Y[i, unique_tokens_index[next_words[i]]] = 2


In [20]:
X

array([[[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ...,  True, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ..., False, False, False],
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False, False, False]],

       [[False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        [False, False, False, ..., False, False, False],
        ...,
        [False, False, False, ...,  True, False, False],
        [False, False, False, ..., False, Fal

In [21]:
Y

array([[False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       ...,
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False],
       [False, False, False, ..., False, False, False]])

In [22]:
model = Sequential()
model.add(LSTM(128 , input_shape=(n_words , len(unique_tokens)) , return_sequences=True))
model.add(LSTM(128 ))
model.add(Dense(len(unique_tokens)))
model.add(Activation("softmax"))

In [25]:
model.compile(
    loss="categorical_crossentropy",
    optimizer=RMSprop(learning_rate=0.01),
    metrics=["accuracy"]
)
model.fit(X,Y,batch_size=128 ,epochs = 40, shuffle ="True")

Epoch 1/40
2/2 [==============================] - 4s 33ms/step - loss: 0.9701 - accuracy: 0.7123
Epoch 2/40
2/2 [==============================] - 0s 33ms/step - loss: 3.1246 - accuracy: 0.2603
Epoch 3/40
2/2 [==============================] - 0s 31ms/step - loss: 2.4924 - accuracy: 0.3699
Epoch 4/40
2/2 [==============================] - 0s 30ms/step - loss: 0.9762 - accuracy: 0.6507
Epoch 5/40
2/2 [==============================] - 0s 31ms/step - loss: 0.6404 - accuracy: 0.7740
Epoch 6/40
2/2 [==============================] - 0s 33ms/step - loss: 0.7710 - accuracy: 0.7466
Epoch 7/40
2/2 [==============================] - 0s 30ms/step - loss: 0.3863 - accuracy: 0.8836
Epoch 8/40
2/2 [==============================] - 0s 30ms/step - loss: 0.2896 - accuracy: 0.9521
Epoch 9/40
2/2 [==============================] - 0s 30ms/step - loss: 0.1794 - accuracy: 0.9932
Epoch 10/40
2/2 [==============================] - 0s 33ms/step - loss: 0.1420 - accuracy: 0.9932
Epoch 11/40
2/2 [============

In [26]:
model.save("mymodel.h5")

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [27]:
model = load_model("mymodel.h5")

In [51]:


# Sample text for training
partial_text = "He will have to look into this thing and he will be a man."


# Fit the model
model.fit(
    X, Y,
    batch_size=128,
    epochs=40,
    shuffle=True
)

def predict_next_word(input_text, n_best, model, n_words, unique_tokens, token_to_index, index_to_token):
    # Ensure the input text is in lowercase
    input_text = input_text.lower()

    # Initialize the input array
    X = np.zeros((1, n_words, len(unique_tokens)), dtype=bool)

    # Tokenize and prepare the input sequence
    tokens = input_text.split()
    for i, word in enumerate(tokens[-n_words:]):
        if word in token_to_index:
            X[0, i, token_to_index[word]] = 1

    # Get predictions from the model
    predictions = model.predict(X)[0]

    # Return the top n_best predictions
    best_indices = np.argpartition(predictions, -n_best)[-n_best:]

    return best_indices

# Example usage
input_text = "He will have to look into this thing and he"
n_best = 1

# Call the function with the required parameters
possible_indices = predict_next_word(input_text, n_best, model, n_words, unique_tokens, token_to_index, index_to_token)

# Decode the predicted word(s)
predicted_words = [index_to_token[idx] for idx in possible_indices]
print(f"Next predicted word(s): {predicted_words}")


Epoch 1/40
1/1 [==============================] - 0s 18ms/step - loss: 0.0047 - accuracy: 1.0000
Epoch 2/40
1/1 [==============================] - 0s 16ms/step - loss: 0.0043 - accuracy: 1.0000
Epoch 3/40
1/1 [==============================] - 0s 13ms/step - loss: 0.0040 - accuracy: 1.0000
Epoch 4/40
1/1 [==============================] - 0s 13ms/step - loss: 0.0038 - accuracy: 1.0000
Epoch 5/40
1/1 [==============================] - 0s 13ms/step - loss: 0.0035 - accuracy: 1.0000
Epoch 6/40
1/1 [==============================] - 0s 14ms/step - loss: 0.0033 - accuracy: 1.0000
Epoch 7/40
1/1 [==============================] - 0s 13ms/step - loss: 0.0031 - accuracy: 1.0000
Epoch 8/40
1/1 [==============================] - 0s 14ms/step - loss: 0.0029 - accuracy: 1.0000
Epoch 9/40
1/1 [==============================] - 0s 15ms/step - loss: 0.0027 - accuracy: 1.0000
Epoch 10/40
1/1 [==============================] - 0s 13ms/step - loss: 0.0026 - accuracy: 1.0000
Epoch 11/40
1/1 [============

In [52]:
def generate_text(input_text, n_words, creativity=3):
    word_sequence = input_text.split()
    current = 0
    for _ in range(n_words):
        sub_sequence = " ".join(tokenizer.tokenize(" ".join(word_sequence).lower())[current:current+n_words])
        try:
            choice = unique_tokens[random.choice(predict_next_word(sub_sequence, creativity))]
        except:
            choice = random.choice(unique_tokens)
        word_sequence.append(choice)
        current += 1
    return " ".join(word_sequence)

In [53]:
generate_text("I will have to look into this thing because I", 100, 10)

'I will have to look into this thing because I thing be be thing man to look look a a this into look thing be be he man a be he a will look he a thing into into will will a have a thing will will be thing into and have be and to and into he be this this to have be a look man he look into this this thing this he thing to have into he have look he thing this be have a will and he man will be will into a to a into be have have will thing man thing have will be'

In [54]:
generate_text("The president of the United States announced yesterday that he", 100, 10)

'The president of the United States announced yesterday that he to man to a be will will into this this thing this and a into he this look and to will be man and be this this and to have a thing man this will to have he man have and this a this a be this into be he and a a into he he be a to will and into this a will this this he and be this into this be into he will into he this look look man he a will to be into into this this and be be have thing into into and'